In [1]:
import os
import django
from asgiref.sync import sync_to_async

# Django 설정 파일 지정
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'TJ_web.settings')

# Django 환경 설정 초기화
django.setup()

# 모델 불러오기
from steam.models import GameInfo, GameReview
import pandas as pd

# game_info.csv 파일 읽기
game_info_df = pd.read_csv('game_info_final.csv')

# 비동기 함수 정의
async def load_game_info():
    for _, row in game_info_df.iterrows():
        game_info = GameInfo(
            title=row['game_name'], 
            genres=row['game_genres'],
            image=row['game_image'],
            price=row['game_price'],  
            release_date=row['game_release_date'],
            mean_playtime=row['playtime_at_review'], 
        )
        await sync_to_async(game_info.save)()  # ORM의 save 작업을 비동기 함수로 변환

# game_review.csv 파일 읽기
game_review_df = pd.read_csv('game_review.csv')

# 비동기 함수 정의
async def load_game_review():
    for _, row in game_review_df.iterrows():
        try:
            game_info = await sync_to_async(GameInfo.objects.get)(title=row['game_name'])  # ORM의 get 작업을 비동기 함수로 변환
            game_review = GameReview(
                game_info=game_info,  # ForeignKey로 참조하는 GameInfo 레코드
                content=row['review_text'],
                recommend=row['review_score'],
                playtime=row['playtime_at_review'],
                created_at=row['timestamp_created'],
            )
            await sync_to_async(game_review.save)()  # ORM의 save 작업을 비동기 함수로 변환
        except GameInfo.DoesNotExist:
            print(f"GameInfo에 해당 게임이 존재하지 않습니다: {row['game_name']}")

# 비동기 메인 함수 실행
import asyncio
async def main():
    await load_game_info()
    await load_game_review()

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop